# How to Search for Catalog Data and IDs with lksearch

`lksearch` enables you to search through the catalogs associated with each mission, and Gaia. You can query:

- The Kepler Input Catalog
- The Ecliptic Plane Input Catalog
- The TESS Input Catalog
- Gaia EDR3.

`lksearch` has three main pieces of functionality - two ways of retrieving data from a catalog, and one for looking up catalog IDs.

These are:
   
- **A position cone-search** that searches for sources within a given radius from some position using `QueryPosition`
- **An exact ID lookup**, retrieving catalog rows where the catalog ID matches the search ID using `QueryID`
- **A way of looking up catalog IDs** given a common name or another catalog ID with `IDLookup`. 

`lksearch` also has the ability to crossmatch targets when doing an exact ID lookup between the TIC, KIC, and Gaia EDR3 catalogs because this crossmatch already exists in the TIC.  

`lksearch` will propagate the proper motion for you, and can optionally return a SkyCoordinate either directly from the Search or by using the `to_skycoord` property from the results table.  

Below are examples of each query.

## Searching by Position

In [1]:
from lksearch import CatalogSearch
from astropy.coordinates import SkyCoord
from astropy.time import Time
import astropy.units as u

Before we search we need a sky position to search. `lksearch` catalog queries use `astropy`'s `SkyCoord` as an input. We can either make one ourselves with an RA and Dec, or we can use the built in `from_name` method.

In [2]:
c = SkyCoord.from_name("Kepler-10")

Now we have a position we want to query, we can query each catalog. For example, here is the query for KIC

In [3]:
CatalogSearch.QueryPosition(c, catalog="kic", max_results=5)

ID,RA,Dec,Separation,Relative_Flux,pmRA,pmDE,Kepmag,Teff,logg
,,,,,mas / yr,mas / yr,mag,K,log(cm.s**-2)
str12,float64,float64,float64,float32,float64,float64,float32,int32,float32
KIC 11904151,285.67938,50.241299,0.0,1.0,0.0,0.0,10.961,5491,4.467
KIC 11904160,285.68076,50.242493,5.345213505938071,0.0066435444,0.0,0.0,16.405,--,--
KIC 11904158,285.68042,50.245716,16.08046158371309,0.0054200143,0.0,0.0,16.626,--,--
KIC 11904156,285.68015,50.250011,31.41325726092946,0.0026717787,-4.0,0.0,17.394,--,--
KIC 11904148,285.67722,50.25901,63.95318581678289,0.017155372,0.0,0.0,15.375,5286,4.640


By default, the catalog will query with an epoch of that is the default for the catalog. You can set the `epoch` keyword to a date in order to get `RA` and `Dec` at that date, given the proper motion.

In [4]:
CatalogSearch.QueryPosition(c, epoch=Time.now(), catalog="kic", max_results=5)

ID,RA,Dec,Separation,Relative_Flux,pmRA,pmDE,Kepmag,Teff,logg
,,,,,mas / yr,mas / yr,mag,K,log(cm.s**-2)
str12,float64,float64,float64,float32,float64,float64,float32,int32,float32
KIC 11904151,285.67938,50.241299,0.0,1.0,0.0,0.0,10.961,5491,4.467
KIC 11904160,285.68076,50.242493,5.345213505938071,0.0066435444,0.0,0.0,16.405,--,--
KIC 11904158,285.68042,50.245716,16.08046158371309,0.0054200143,0.0,0.0,16.626,--,--
KIC 11904156,285.6801067536893,50.25001099999198,31.407796176187023,0.0026717787,-4.0,0.0,17.394,--,--
KIC 11904148,285.67722,50.25901000000001,63.95318581678289,0.017155372,0.0,0.0,15.375,5286,4.640


You can also pass a radius for your query. If you query each catalog directly you can set your query radius in units of pixels, otherwise you can use arcseconds.

In [5]:
CatalogSearch.QueryPosition(c, epoch=Time.now(), radius=30 * u.arcsec, catalog="kic")

ID,RA,Dec,Separation,Relative_Flux,pmRA,pmDE,Kepmag,Teff,logg
,,,,,mas / yr,mas / yr,mag,K,log(cm.s**-2)
str12,float64,float64,float64,float32,float64,float64,float32,int32,float32
KIC 11904151,285.67938,50.241299,0.0,1.0,0.0,0.0,10.961,5491,4.467
KIC 11904160,285.68076,50.242493,5.345213505938071,0.0066435444,0.0,0.0,16.405,--,--
KIC 11904165,285.68317,50.241207,8.732390194683243,0.002659501,0.0,0.0,17.399,--,--
KIC 11904162,285.68124,50.24391599999999,10.348792766445591,0.0015761611,0.0,0.0,17.967,--,--
KIC 11904158,285.68042,50.245716,16.08046158371309,0.0054200143,0.0,0.0,16.626,--,--


You can query TIC

In [6]:
CatalogSearch.QueryPosition(c, epoch=Time.now(), radius=20 * u.arcsec, catalog="tic")

ID,RA,Dec,Separation,Relative_Flux,pmRA,pmDE,TESSmag,Mass,Rad,Teff,logg
,,,,,mas / yr,mas / yr,mag,solMass,solRad,K,log(cm.s**-2)
str14,float64,float64,float64,float32,float64,float64,float32,float32,float64,float64,float32
TIC 377780790,285.6792229593456,50.241592310484634,nan,915.37714,-18.394,41.448,10.477,1.017,1.090,5706.0,4.3707
TIC 1717218059,nan,nan,nan,1.0,-2.227,-1.204,17.881,1.070,0.810,5895.0,4.6506


EPIC (Note Kepler-10 isn't inside EPIC, because it's outside the ecliptic plane!)

In [7]:
CatalogSearch.QueryPosition(c, catalog="epic")

RAJ2000,DEJ2000,pmRA,pmDE,Plx,K2mag,logg,Teff,Rad,Mass,RA,Dec,Separation,Relative_Flux
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


And Gaia in the same ways

In [8]:
CatalogSearch.QueryPosition(
    c, epoch=Time.now(), radius=40 * u.arcsecond, catalog="gaiadr3"
)

ID,RA,Dec,Separation,Relative_Flux,pmRA,pmDE,BPmag,Gmag,RPmag,Teff,logg
,,,,,mas / yr,mas / yr,mag,mag,mag,K,log(cm.s**-2)
str28,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
Gaia DR3 2132155017099178624,285.6793511035652,50.24140817434181,0.0,1.0,-18.483,41.382,11.253505,10.920031,10.418263,5595.8,4.3544
Gaia DR3 2132155051458917632,285.68021338390014,50.24994176562508,30.78499970152597,0.0016912912421145657,-1.519,-1.203,18.209782,17.849485,17.317846,5424.7,4.6774
Gaia DR3 2132155051458918144,285.6859229038892,50.249918851499665,34.170405504656244,0.005578085926713943,9.402,5.814,17.213984,16.553818,15.778805,--,--


Optionally, you can return the output of the search as an `astropy.coordinates.SkyCoord` object

In [9]:
CatalogSearch.QueryPosition(
    c,
    epoch=Time.now(),
    radius=40 * u.arcsecond,
    catalog="gaiadr3",
    return_skycoord=True,
)

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    [(285.6793511 , 50.24140817,  186.22667509),
     (285.68021338, 50.24994177, 7132.66761762),
     (285.6859229 , 50.24991885, 1124.73287589)]
 (pm_ra_cosdec, pm_dec, radial_velocity) in (mas / yr, mas / yr, km / s)
    [(-18.48303962, 41.38198231, 7.81382436e-05),
     ( -1.51899991, -1.20300012, 5.47033878e-06),
     (  9.40200283,  5.81399542, 2.80752310e-05)]>

You can also sub-select from the results table and then convert to a SkyCoordinate

In [10]:
result = CatalogSearch.QueryPosition(
    c, epoch=Time.now(), radius=40 * u.arcsecond, catalog="gaiadr3"
)
result

ID,RA,Dec,Separation,Relative_Flux,pmRA,pmDE,BPmag,Gmag,RPmag,Teff,logg
,,,,,mas / yr,mas / yr,mag,mag,mag,K,log(cm.s**-2)
str28,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
Gaia DR3 2132155017099178624,285.6793511035652,50.241408174341814,0.0,1.0,-18.483,41.382,11.253505,10.920031,10.418263,5595.8,4.3544
Gaia DR3 2132155051458917632,285.68021338390014,50.24994176562508,30.784999701480263,0.0016912912421145657,-1.519,-1.203,18.209782,17.849485,17.317846,5424.7,4.6774
Gaia DR3 2132155051458918144,285.6859229038892,50.249918851499665,34.17040550463571,0.005578085926713943,9.402,5.814,17.213984,16.553818,15.778805,--,--


In [11]:
result.to_SkyCoord()

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    [(285.6793511 , 50.24140817, 10000000.),
     (285.68021338, 50.24994177, 10000000.),
     (285.6859229 , 50.24991885, 10000000.)]
 (pm_ra_cosdec, pm_dec) in mas / yr
    [(-18.483, 41.382), ( -1.519, -1.203), (  9.402,  5.814)]>

In [12]:
result.to_SkyCoord()

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    [(285.6793511 , 50.24140817, 10000000.),
     (285.68021338, 50.24994177, 10000000.),
     (285.6859229 , 50.24991885, 10000000.)]
 (pm_ra_cosdec, pm_dec) in mas / yr
    [(-18.483, 41.382), ( -1.519, -1.203), (  9.402,  5.814)]>

## Searching by Catalog ID

For TIC, KIC, EPIC, and Gaia DR3 - if we know the Catalog ID of a source we can use `QueryID` to return the data from that catalog for that id.  For example, Kepler-10 above is also known as TIC 377780790. 

In [13]:
CatalogSearch.QueryID("TIC 377780790")

TIC,RAJ2000,DEJ2000,HIP,TYC,UCAC4,_2MASS,objID,WISEA,GAIA,APASS,KIC,S_G,Ref,r_Pos,pmRA,e_pmRA,pmDE,e_pmDE,r_pm,Plx,e_Plx,r_Plx,GLON,GLAT,ELON,ELAT,Bmag,e_Bmag,Vmag,e_Vmag,umag,e_umag,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,q_2MASS,W1mag,e_W1mag,W2mag,e_W2mag,W3mag,e_W3mag,W4mag,e_W4mag,Gmag,e_Gmag,Tmag,e_Tmag,f_Tmag,Flag,Teff,s_Teff,logg,s_logg,__M_H_,e__M_H_,Rad,s_Rad,Mass,s_Mass,rho,s_rho,LClass,Lum,s_Lum,Dist,s_Dist,E_B-V_,s_E_B-V_,Ncont,Rcont,Disp,m_TIC,Prior,e_E_B-V_,E_E_B-V_,f_E_B-V_,e_Mass,E_Mass,e_Rad,E_Rad,e_rho,E_rho,e_logg,E_logg,e_Lum,E_Lum,e_Dist,E_Dist,r_Dist,e_Teff,E_Teff,r_Teff,BPmag,e_BPmag,RPmag,e_RPmag,q_Gaia,r_Vmag,r_Bmag,Clist,e_RAJ2000,e_DEJ2000,RAOdeg,DEOdeg,e_RAOdeg,e_DEOdeg,RadFl,WDFl,ID
,deg,deg,,,,,,,,,,,,,mas / yr,mas / yr,mas / yr,mas / yr,,mas,mas,,deg,deg,deg,deg,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,,K,K,log(cm.s**-2),log(cm.s**-2),log(Sun),log(Sun),Rsun,Rsun,Msun,Msun,Sun,Sun,,Lsun,Lsun,pc,pc,mag,mag,,,,,,mag,mag,,Msun,Msun,Rsun,Rsun,Sun,Sun,log(cm.s**-2),log(cm.s**-2),Lsun,Lsun,pc,pc,,K,K,,mag,mag,mag,mag,,,,,mas,mas,deg,deg,mas,mas,,,
int64,float64,float64,int32,object,object,object,int64,object,int64,int32,int32,object,object,object,float64,float32,float64,float32,object,float64,float32,object,float64,float64,float64,float64,float32,float32,float32,float32,float32,float64,float32,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,object,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float32,float32,object,object,float32,float32,float32,float32,float32,float32,float64,float32,float32,float32,float64,float64,object,float64,float64,float64,float64,float32,float32,int32,float64,object,int64,float64,float32,float32,object,float32,float32,float32,float32,float64,float64,float32,float32,float64,float64,float64,float64,object,float32,float32,object,float64,float64,float64,float64,int16,object,object,object,float64,float64,float64,float64,float64,float64,int16,int16,int32
377780790,285.67942179238,50.24130576642,--,,702-062695,19024305+5014286,--,J190243.03+501429.1,2132155017099178624,--,11904151,STAR,tmgaia2,tmgaia2,-18.394,0.045,41.448,0.046,gaia2,5.3619,0.0233,gaia2,80.48973434189,18.81612807429,303.70102580688,71.84987860932,12.258,0.067,11.043,0.023,--,--,--,--,--,--,--,--,--,--,9.889,0.022,9.563,0.025,9.496,0.021,AAA-222-111-000-0-0,9.44,0.023,9.491,0.02,9.443,0.029,9.403,0.482,10.9191,0.000318,10.477,0.006,rered,spect,5706.0,141.0,4.3707,0.0854,-0.12,0.1,1.09,0.061,1.017,0.128,0.7859,0.1845,DWARF,1.13414,0.03284,185.506,0.808,0.0107,0.0037,223,0.0101,,--,0.002668,0.0028,0.0046,panstarrs,0.119,0.138,0.061,0.061,0.1571,0.2119,0.1042,0.0666,0.03598,0.0297,0.805,0.811,bj2018,125.2,156.8,spect,11.2713,0.001006,10.4273,0.000585,1,ucac4,bpbj,,0.767293,0.711508,285.67929796407,50.24148422359,0.020474,0.024043,1,0,237087497


... and KIC 11904151

In [14]:
CatalogSearch.QueryID("KIC 11904151")

KIC,RAJ2000,DEJ2000,pmRA,pmDE,Plx,umag,gmag,rmag,imag,zmag,grmag,d51mag,Jmag,Hmag,Kmag,kepmag,_2Mkey,_2Mname,SCP-ID,SCPkey,catkey,alc,altID,sg,v,cq,pq,aq,fv,Teff,logg,__Fe_H_,E_B-V_,Av,R_
,deg,deg,mas / yr,mas / yr,mas,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,,,,,,,,,,,,,K,log(cm.s**-2),log(Sun),mag,mag,Rsun
int32,float64,float64,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int32,object,int32,int32,int32,int16,int32,int16,int16,object,int16,int16,int16,int32,float32,float32,float32,float32,float32
11904151,285.67938,50.241299,0.0,0.0,--,--,11.388,10.92,10.778,10.729,--,11.191,9.888,9.563,9.496,10.961,1065107578,J19024305+5014286,1065107578,--,11732614,3,-1058152,0,0,SCP,5,6,2,5491,4.467,-0.812,0.031,0.095,0.98


... and Gaia DR3 2132155017099178624

In [15]:
CatalogSearch.QueryID("Gaia DR3 2132155017099178624")

DR3Name,RA_ICRS,DE_ICRS,SolID,Source,RandomI,e_RA_ICRS,e_DE_ICRS,Plx,e_Plx,RPlx,PM,pmRA,e_pmRA,pmDE,e_pmDE,RADEcor,RAPlxcor,RApmRAcor,RApmDEcor,DEPlxcor,DEpmRAcor,DEpmDEcor,PlxpmRAcor,PlxpmDEcor,pmRApmDEcor,NAL,NAC,NgAL,NbAL,gofAL,chi2AL,epsi,sepsi,Solved,APF,nueff,pscol,e_pscol,RApscolCorr,DEpscolCorr,PlxpscolCorr,pmRApscolCorr,pmDEpscolCorr,MatchObsA,Nper,amax,MatchObs,NewMatchObs,MatchObsrm,IPDgofha,IPDgofhp,IPDfmp,IPDfow,RUWE,SDSk1,SDSk2,SDSk3,SDSk4,SDMk1,SDMk2,SDMk3,SDMk4,Dup,o_Gmag,FG,e_FG,RFG,Gmag,e_Gmag,o_BPmag,FBP,e_FBP,RFBP,BPmag,e_BPmag,o_RPmag,FRP,e_FRP,RFRP,RPmag,e_RPmag,E_BP_RP_,NBPcont,NBPblend,NRPcont,NRPblend,Mode,BP-RP,BP-G,G-RP,RV,e_RV,n_RV,o_RV,o_RVd,RVNper,RVS_N,RVgof,RVchi2,RVTdur,RVamp,RVtempTeff,RVtemplogg,RVtemp_Fe_H_,Vatmparam,Vbroad,e_Vbroad,o_Vbroad,GRVSmag,e_GRVSmag,o_GRVSmag,RVSS_N,VarFlag,GLON,GLAT,ELON,ELAT,QSO,Gal,NSS,XPcont,XPsamp,RVS,EpochPh,EpochRV,MCMCGSP,MCMCMSC,And,PQSO,PGal,PSS,Teff,b_Teff,B_Teff,logg,b_logg,B_logg,__Fe_H_,b__Fe_H_,B__Fe_H_,Dist,b_Dist,B_Dist,A0,b_A0,B_A0,AG,b_AG,B_AG,E_BP-RP_,b_E_BP-RP_,B_E_BP-RP_,Lib,HIP,dHIP,nHIP,f_HIP,PS1coid,PS1,dPS1,nPS1,mPS1,f_PS1,SDSS13coid,SDSS13,dSDSS13,nSDSS13,mSDSS13,f_SDSS13,SKYM2,dSKYM2,nSKYM2,mSKYM2,f_SKYM2,TYC2,dTYC2,f_TYC2,TYC2moid,nTYC2,URAT1,dURAT1,f_URAT1,URAT1oid,nURAT1,mURAT1,AllWISE,dAllWISE,f_AllWISE,AllWISEoid,nAllWISE,mAllWISE,APASS9coid,APASS9,dAPASS9,nAPASS9,mAPASS9,f_APASS9,GSC23,dGSC23,f_GSC23,GSC23coid,nGSC23,mGSC23,RAVE5,dRAVE5,f_RAVE5,RAVE5coid,nRAVE5,_2MASS,d2MASS,f_2MASS,_2MASScoid,n2MASS,m2MASS,RAVE6,dRAVE6,f_RAVE6,RAVE6oid,nRAVE6,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,RADEcorJ2000
,deg,deg,,,,mas,mas,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / um,1 / um,1 / um,,,,,,,,mas,,,,,deg,,,,,,,,,,,deg,,,,,,mag,mag,,,,,mag,mag,,,,,mag,mag,mag,,,,,,mag,mag,mag,km / s,km / s,,,,,,,,d,km / s,K,log(cm.s**-2),,,km / s,km / s,,mag,mag,,,,deg,deg,deg,deg,,,,,,,,,,,,,,,K,K,K,log(cm.s**-2),log(cm.s**-2),log(cm.s**-2),,,,pc,pc,pc,mag,mag,mag,mag,mag,mag,mag,mag,mag,,,mas,,,,,mas,,,,,,mas,,,,,mas,,,,,mas,,,,,mas,,,,,,mas,,,,,,,mas,,,,,mas,,,,,,mas,,,,,mas,,,,,,mas,,,,deg,deg,mas,mas,
object,float64,float64,int64,int64,int64,float32,float32,float64,float32,float64,float64,float64,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,float64,float64,float64,float64,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,float64,int16,int16,int16,float64,float64,int16,int16,float32,float64,float64,float64,float64,float64,float32,float32,float32,int16,int16,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,float64,float64,int16,int16,int16,int16,int16,float64,float64,float64,float32,float32,int16,int16,int16,int16,float64,float64,float64,float64,float32,float32,float32,float32,int16,float64,float64,int16,float64,float64,int16,float64,object,float64,float64,float64,float64,int16,int16,int16,int16,int16,int16,int16,int16,int16,int16,int16,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,object,int32,float64,int16,int16,int64,int64,float64,int16,int16,int16,int32,int64,float64,int16,int16,int16,int32,float64,int16,int16,int16,object,float64,int16,int32,int16,object,float64,int16,int32,int16,int16,object,float64,int16,int32,int16,int16,int32,int32,float64,int16,int16,int16,object,float64,int16,int32,int16,int16,object,float64,int16,float64,int16,object,float64,int16,int32,int16,int16,object,float64,int16,float64,int16,float64,float64,float64,float64,float32
Gaia DR3 2132155017099178624,285.67929401534,50.24148992495,1636148068921376768,2132155017099178624,712720541,0.0097,0.0105,5.3698,0.0103,521.1521,45.322,-18.483,0.012,41.382,0.013,-0.0139,0.0622,-0.0681,-0.1702,0.0472,-0.1583,-0.201,-0.0394,-

The TIC catalog has been cross-matched with the GAIA DR3 and KIC catalogs.  `lksearch` can use this cross-matching to retrieve TIC/KIC/GAIA catalog info for any given TIC/KIC/GaiaDR3 source ID.  The EPIC catalog is not included in this cross-matching.  

For Example, if we want the TIC info for Kepler-10, but only have the GAIA DR3Name above, we can get that like so:

In [16]:
CatalogSearch.QueryID("Gaia DR3 2132155017099178624", catalog="tic")

TIC,RAJ2000,DEJ2000,HIP,TYC,UCAC4,_2MASS,objID,WISEA,GAIA,APASS,KIC,S_G,Ref,r_Pos,pmRA,e_pmRA,pmDE,e_pmDE,r_pm,Plx,e_Plx,r_Plx,GLON,GLAT,ELON,ELAT,Bmag,e_Bmag,Vmag,e_Vmag,umag,e_umag,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,q_2MASS,W1mag,e_W1mag,W2mag,e_W2mag,W3mag,e_W3mag,W4mag,e_W4mag,Gmag,e_Gmag,Tmag,e_Tmag,f_Tmag,Flag,Teff,s_Teff,logg,s_logg,__M_H_,e__M_H_,Rad,s_Rad,Mass,s_Mass,rho,s_rho,LClass,Lum,s_Lum,Dist,s_Dist,E_B-V_,s_E_B-V_,Ncont,Rcont,Disp,m_TIC,Prior,e_E_B-V_,E_E_B-V_,f_E_B-V_,e_Mass,E_Mass,e_Rad,E_Rad,e_rho,E_rho,e_logg,E_logg,e_Lum,E_Lum,e_Dist,E_Dist,r_Dist,e_Teff,E_Teff,r_Teff,BPmag,e_BPmag,RPmag,e_RPmag,q_Gaia,r_Vmag,r_Bmag,Clist,e_RAJ2000,e_DEJ2000,RAOdeg,DEOdeg,e_RAOdeg,e_DEOdeg,RadFl,WDFl,ID
,deg,deg,,,,,,,,,,,,,mas / yr,mas / yr,mas / yr,mas / yr,,mas,mas,,deg,deg,deg,deg,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,,K,K,log(cm.s**-2),log(cm.s**-2),log(Sun),log(Sun),Rsun,Rsun,Msun,Msun,Sun,Sun,,Lsun,Lsun,pc,pc,mag,mag,,,,,,mag,mag,,Msun,Msun,Rsun,Rsun,Sun,Sun,log(cm.s**-2),log(cm.s**-2),Lsun,Lsun,pc,pc,,K,K,,mag,mag,mag,mag,,,,,mas,mas,deg,deg,mas,mas,,,
int64,float64,float64,int32,object,object,object,int64,object,int64,int32,int32,object,object,object,float64,float32,float64,float32,object,float64,float32,object,float64,float64,float64,float64,float32,float32,float32,float32,float32,float64,float32,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,object,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float32,float32,object,object,float32,float32,float32,float32,float32,float32,float64,float32,float32,float32,float64,float64,object,float64,float64,float64,float64,float32,float32,int32,float64,object,int64,float64,float32,float32,object,float32,float32,float32,float32,float64,float64,float32,float32,float64,float64,float64,float64,object,float32,float32,object,float64,float64,float64,float64,int16,object,object,object,float64,float64,float64,float64,float64,float64,int16,int16,int32
377780790,285.67942179238,50.24130576642,--,,702-062695,19024305+5014286,--,J190243.03+501429.1,2132155017099178624,--,11904151,STAR,tmgaia2,tmgaia2,-18.394,0.045,41.448,0.046,gaia2,5.3619,0.0233,gaia2,80.48973434189,18.81612807429,303.70102580688,71.84987860932,12.258,0.067,11.043,0.023,--,--,--,--,--,--,--,--,--,--,9.889,0.022,9.563,0.025,9.496,0.021,AAA-222-111-000-0-0,9.44,0.023,9.491,0.02,9.443,0.029,9.403,0.482,10.9191,0.000318,10.477,0.006,rered,spect,5706.0,141.0,4.3707,0.0854,-0.12,0.1,1.09,0.061,1.017,0.128,0.7859,0.1845,DWARF,1.13414,0.03284,185.506,0.808,0.0107,0.0037,223,0.0101,,--,0.002668,0.0028,0.0046,panstarrs,0.119,0.138,0.061,0.061,0.1571,0.2119,0.1042,0.0666,0.03598,0.0297,0.805,0.811,bj2018,125.2,156.8,spect,11.2713,0.001006,10.4273,0.000585,1,ucac4,bpbj,,0.767293,0.711508,285.67929796407,50.24148422359,0.020474,0.024043,1,0,237087497


We can also supply an integer ID, but if so we MUST specify what catalog to search

In [17]:
CatalogSearch.QueryID("2132155017099178624", catalog="gaiadr3")

DR3Name,RA_ICRS,DE_ICRS,SolID,Source,RandomI,e_RA_ICRS,e_DE_ICRS,Plx,e_Plx,RPlx,PM,pmRA,e_pmRA,pmDE,e_pmDE,RADEcor,RAPlxcor,RApmRAcor,RApmDEcor,DEPlxcor,DEpmRAcor,DEpmDEcor,PlxpmRAcor,PlxpmDEcor,pmRApmDEcor,NAL,NAC,NgAL,NbAL,gofAL,chi2AL,epsi,sepsi,Solved,APF,nueff,pscol,e_pscol,RApscolCorr,DEpscolCorr,PlxpscolCorr,pmRApscolCorr,pmDEpscolCorr,MatchObsA,Nper,amax,MatchObs,NewMatchObs,MatchObsrm,IPDgofha,IPDgofhp,IPDfmp,IPDfow,RUWE,SDSk1,SDSk2,SDSk3,SDSk4,SDMk1,SDMk2,SDMk3,SDMk4,Dup,o_Gmag,FG,e_FG,RFG,Gmag,e_Gmag,o_BPmag,FBP,e_FBP,RFBP,BPmag,e_BPmag,o_RPmag,FRP,e_FRP,RFRP,RPmag,e_RPmag,E_BP_RP_,NBPcont,NBPblend,NRPcont,NRPblend,Mode,BP-RP,BP-G,G-RP,RV,e_RV,n_RV,o_RV,o_RVd,RVNper,RVS_N,RVgof,RVchi2,RVTdur,RVamp,RVtempTeff,RVtemplogg,RVtemp_Fe_H_,Vatmparam,Vbroad,e_Vbroad,o_Vbroad,GRVSmag,e_GRVSmag,o_GRVSmag,RVSS_N,VarFlag,GLON,GLAT,ELON,ELAT,QSO,Gal,NSS,XPcont,XPsamp,RVS,EpochPh,EpochRV,MCMCGSP,MCMCMSC,And,PQSO,PGal,PSS,Teff,b_Teff,B_Teff,logg,b_logg,B_logg,__Fe_H_,b__Fe_H_,B__Fe_H_,Dist,b_Dist,B_Dist,A0,b_A0,B_A0,AG,b_AG,B_AG,E_BP-RP_,b_E_BP-RP_,B_E_BP-RP_,Lib,HIP,dHIP,nHIP,f_HIP,PS1coid,PS1,dPS1,nPS1,mPS1,f_PS1,SDSS13coid,SDSS13,dSDSS13,nSDSS13,mSDSS13,f_SDSS13,SKYM2,dSKYM2,nSKYM2,mSKYM2,f_SKYM2,TYC2,dTYC2,f_TYC2,TYC2moid,nTYC2,URAT1,dURAT1,f_URAT1,URAT1oid,nURAT1,mURAT1,AllWISE,dAllWISE,f_AllWISE,AllWISEoid,nAllWISE,mAllWISE,APASS9coid,APASS9,dAPASS9,nAPASS9,mAPASS9,f_APASS9,GSC23,dGSC23,f_GSC23,GSC23coid,nGSC23,mGSC23,RAVE5,dRAVE5,f_RAVE5,RAVE5coid,nRAVE5,_2MASS,d2MASS,f_2MASS,_2MASScoid,n2MASS,m2MASS,RAVE6,dRAVE6,f_RAVE6,RAVE6oid,nRAVE6,RAJ2000,DEJ2000,e_RAJ2000,e_DEJ2000,RADEcorJ2000
,deg,deg,,,,mas,mas,mas,mas,,mas / yr,mas / yr,mas / yr,mas / yr,mas / yr,,,,,,,,,,,,,,,,,mas,,,,1 / um,1 / um,1 / um,,,,,,,,mas,,,,,deg,,,,,,,,,,,deg,,,,,,mag,mag,,,,,mag,mag,,,,,mag,mag,mag,,,,,,mag,mag,mag,km / s,km / s,,,,,,,,d,km / s,K,log(cm.s**-2),,,km / s,km / s,,mag,mag,,,,deg,deg,deg,deg,,,,,,,,,,,,,,,K,K,K,log(cm.s**-2),log(cm.s**-2),log(cm.s**-2),,,,pc,pc,pc,mag,mag,mag,mag,mag,mag,mag,mag,mag,,,mas,,,,,mas,,,,,,mas,,,,,mas,,,,,mas,,,,,mas,,,,,,mas,,,,,,,mas,,,,,mas,,,,,,mas,,,,,mas,,,,,,mas,,,,deg,deg,mas,mas,
object,float64,float64,int64,int64,int64,float32,float32,float64,float32,float64,float64,float64,float32,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,int16,int16,float64,float64,float64,float64,int16,int16,float32,float32,float32,float32,float32,float32,float32,float32,int16,int16,float64,int16,int16,int16,float64,float64,int16,int16,float32,float64,float64,float64,float64,float64,float32,float32,float32,int16,int16,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,float64,float64,int16,int16,int16,int16,int16,float64,float64,float64,float32,float32,int16,int16,int16,int16,float64,float64,float64,float64,float32,float32,float32,float32,int16,float64,float64,int16,float64,float64,int16,float64,object,float64,float64,float64,float64,int16,int16,int16,int16,int16,int16,int16,int16,int16,int16,int16,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float64,float32,float32,float32,float32,float32,float32,float32,float32,float32,object,int32,float64,int16,int16,int64,int64,float64,int16,int16,int16,int32,int64,float64,int16,int16,int16,int32,float64,int16,int16,int16,object,float64,int16,int32,int16,object,float64,int16,int32,int16,int16,object,float64,int16,int32,int16,int16,int32,int32,float64,int16,int16,int16,object,float64,int16,int32,int16,int16,object,float64,int16,float64,int16,object,float64,int16,int32,int16,int16,object,float64,int16,float64,int16,float64,float64,float64,float64,float32
Gaia DR3 2132155017099178624,285.67929401534,50.24148992495,1636148068921376768,2132155017099178624,712720541,0.0097,0.0105,5.3698,0.0103,521.1521,45.322,-18.483,0.012,41.382,0.013,-0.0139,0.0622,-0.0681,-0.1702,0.0472,-0.1583,-0.201,-0.0394,-

If we specify an integer and wish to use our limited cross-match, we can do that by specifying both the `input_catalog` and `catalog` parameters

In [18]:
CatalogSearch.QueryID("2132155017099178624", catalog="tic", input_catalog="gaiadr3")

TIC,RAJ2000,DEJ2000,HIP,TYC,UCAC4,_2MASS,objID,WISEA,GAIA,APASS,KIC,S_G,Ref,r_Pos,pmRA,e_pmRA,pmDE,e_pmDE,r_pm,Plx,e_Plx,r_Plx,GLON,GLAT,ELON,ELAT,Bmag,e_Bmag,Vmag,e_Vmag,umag,e_umag,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,q_2MASS,W1mag,e_W1mag,W2mag,e_W2mag,W3mag,e_W3mag,W4mag,e_W4mag,Gmag,e_Gmag,Tmag,e_Tmag,f_Tmag,Flag,Teff,s_Teff,logg,s_logg,__M_H_,e__M_H_,Rad,s_Rad,Mass,s_Mass,rho,s_rho,LClass,Lum,s_Lum,Dist,s_Dist,E_B-V_,s_E_B-V_,Ncont,Rcont,Disp,m_TIC,Prior,e_E_B-V_,E_E_B-V_,f_E_B-V_,e_Mass,E_Mass,e_Rad,E_Rad,e_rho,E_rho,e_logg,E_logg,e_Lum,E_Lum,e_Dist,E_Dist,r_Dist,e_Teff,E_Teff,r_Teff,BPmag,e_BPmag,RPmag,e_RPmag,q_Gaia,r_Vmag,r_Bmag,Clist,e_RAJ2000,e_DEJ2000,RAOdeg,DEOdeg,e_RAOdeg,e_DEOdeg,RadFl,WDFl,ID
,deg,deg,,,,,,,,,,,,,mas / yr,mas / yr,mas / yr,mas / yr,,mas,mas,,deg,deg,deg,deg,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,,K,K,log(cm.s**-2),log(cm.s**-2),log(Sun),log(Sun),Rsun,Rsun,Msun,Msun,Sun,Sun,,Lsun,Lsun,pc,pc,mag,mag,,,,,,mag,mag,,Msun,Msun,Rsun,Rsun,Sun,Sun,log(cm.s**-2),log(cm.s**-2),Lsun,Lsun,pc,pc,,K,K,,mag,mag,mag,mag,,,,,mas,mas,deg,deg,mas,mas,,,
int64,float64,float64,int32,object,object,object,int64,object,int64,int32,int32,object,object,object,float64,float32,float64,float32,object,float64,float32,object,float64,float64,float64,float64,float32,float32,float32,float32,float32,float64,float32,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,object,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float32,float32,object,object,float32,float32,float32,float32,float32,float32,float64,float32,float32,float32,float64,float64,object,float64,float64,float64,float64,float32,float32,int32,float64,object,int64,float64,float32,float32,object,float32,float32,float32,float32,float64,float64,float32,float32,float64,float64,float64,float64,object,float32,float32,object,float64,float64,float64,float64,int16,object,object,object,float64,float64,float64,float64,float64,float64,int16,int16,int32
377780790,285.67942179238,50.24130576642,--,,702-062695,19024305+5014286,--,J190243.03+501429.1,2132155017099178624,--,11904151,STAR,tmgaia2,tmgaia2,-18.394,0.045,41.448,0.046,gaia2,5.3619,0.0233,gaia2,80.48973434189,18.81612807429,303.70102580688,71.84987860932,12.258,0.067,11.043,0.023,--,--,--,--,--,--,--,--,--,--,9.889,0.022,9.563,0.025,9.496,0.021,AAA-222-111-000-0-0,9.44,0.023,9.491,0.02,9.443,0.029,9.403,0.482,10.9191,0.000318,10.477,0.006,rered,spect,5706.0,141.0,4.3707,0.0854,-0.12,0.1,1.09,0.061,1.017,0.128,0.7859,0.1845,DWARF,1.13414,0.03284,185.506,0.808,0.0107,0.0037,223,0.0101,,--,0.002668,0.0028,0.0046,panstarrs,0.119,0.138,0.061,0.061,0.1571,0.2119,0.1042,0.0666,0.03598,0.0297,0.805,0.811,bj2018,125.2,156.8,spect,11.2713,0.001006,10.4273,0.000585,1,ucac4,bpbj,,0.767293,0.711508,285.67929796407,50.24148422359,0.020474,0.024043,1,0,237087497


We can also search for multiple IDS from a single catalog and get a table of all of the resulting catalog entries

In [19]:
# Kepler Objects 1-10
idlist = [
    "TIC 399860444",
    "TIC 424865156",
    "TIC 28230919",
    "TIC 350814622",
    "TIC 268823307",
    "TIC 27916356",
    "TIC 121660904",
    "TIC 123233041",
    "TIC 120571842",
    "TIC 377780790",
]
CatalogSearch.QueryID(idlist, catalog="tic")

TIC,RAJ2000,DEJ2000,HIP,TYC,UCAC4,_2MASS,objID,WISEA,GAIA,APASS,KIC,S_G,Ref,r_Pos,pmRA,e_pmRA,pmDE,e_pmDE,r_pm,Plx,e_Plx,r_Plx,GLON,GLAT,ELON,ELAT,Bmag,e_Bmag,Vmag,e_Vmag,umag,e_umag,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,q_2MASS,W1mag,e_W1mag,W2mag,e_W2mag,W3mag,e_W3mag,W4mag,e_W4mag,Gmag,e_Gmag,Tmag,e_Tmag,f_Tmag,Flag,Teff,s_Teff,logg,s_logg,__M_H_,e__M_H_,Rad,s_Rad,Mass,s_Mass,rho,s_rho,LClass,Lum,s_Lum,Dist,s_Dist,E_B-V_,s_E_B-V_,Ncont,Rcont,Disp,m_TIC,Prior,e_E_B-V_,E_E_B-V_,f_E_B-V_,e_Mass,E_Mass,e_Rad,E_Rad,e_rho,E_rho,e_logg,E_logg,e_Lum,E_Lum,e_Dist,E_Dist,r_Dist,e_Teff,E_Teff,r_Teff,BPmag,e_BPmag,RPmag,e_RPmag,q_Gaia,r_Vmag,r_Bmag,Clist,e_RAJ2000,e_DEJ2000,RAOdeg,DEOdeg,e_RAOdeg,e_DEOdeg,RadFl,WDFl,ID
,deg,deg,,,,,,,,,,,,,mas / yr,mas / yr,mas / yr,mas / yr,,mas,mas,,deg,deg,deg,deg,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,,K,K,log(cm.s**-2),log(cm.s**-2),log(Sun),log(Sun),Rsun,Rsun,Msun,Msun,Sun,Sun,,Lsun,Lsun,pc,pc,mag,mag,,,,,,mag,mag,,Msun,Msun,Rsun,Rsun,Sun,Sun,log(cm.s**-2),log(cm.s**-2),Lsun,Lsun,pc,pc,,K,K,,mag,mag,mag,mag,,,,,mas,mas,deg,deg,mas,mas,,,
int64,float64,float64,int32,object,object,object,int64,object,int64,int32,int32,object,object,object,float64,float32,float64,float32,object,float64,float32,object,float64,float64,float64,float64,float32,float32,float32,float32,float32,float64,float32,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,object,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float32,float32,object,object,float32,float32,float32,float32,float32,float32,float64,float32,float32,float32,float64,float64,object,float64,float64,float64,float64,float32,float32,int32,float64,object,int64,float64,float32,float32,object,float32,float32,float32,float32,float64,float64,float32,float32,float64,float64,float64,float64,object,float32,float32,object,float64,float64,float64,float64,int16,object,object,object,float64,float64,float64,float64,float64,float64,int16,int16,int32
27916356,296.83724132289,48.23993319637,--,,692-068460,19472094+4814238,--,J194720.94+481423.9,2086636884980514304,54466301,10874614,STAR,tmgaia2,tmgaia2,0.961,0.025,8.429,0.026,gaia2,1.6747,0.0145,gaia2,81.69266112428,11.35938157922,320.42485523961,67.03995916388,14.051,0.175,13.286,0.08,--,--,--,--,--,--,--,--,--,--,12.001,0.021,11.706,0.019,11.634,0.019,AAA-222-111-000-0-0,11.585,0.022,11.643,0.021,11.494,0.141,8.965,--,13.1714,0.000286,12.688,0.007,rered,spect,5613.0,107.7,4.196,0.076,0.27,0.1,1.315,0.06,0.99,0.132,0.4356,0.0899,DWARF,1.54582,0.04946,587.039,5.0285,0.0397,0.0038,560,0.2498,,--,0.000438,0.0026,0.0051,panstarrs,0.102,0.161,0.046,0.073,0.0609,0.1189,0.089,0.063,0.0644,0.03452,4.986,5.071,bj2018,89.9,125.6,spect,13.5617,0.001563,12.6346,0.000856,1,ucac4,apassdr9,,0.431661,0.4011,296.83724753744,48.23996948867,0.011393,0.014386,1,0,338972619
28230919,297.70936440263,48.08086037921,97657,3561-02092-1,691-070316,19505021+4804508,--,,2086512227851023872,54470133,10748390,STAR,tmgaia2,tmgaia2,125.984,0.041,232.988,0.045,gaia2,26.4508,0.0234,gaia2,81.83370338271,10.77238081693,321.54803605747,66.62944166359,10.556,0.141,9.46,0.03,--,--,--,--,--,--,--,--,--,--,7.608,0.029,7.131,0.021,7.009,0.02,AAA-111-111-000-0-0,--,--,--,--,--,--,--,--,9.15098,0.000293,8.508,0.006,gbprp,spect,4777.6,112.9,4.5631,0.0865,0.2996,0.0527,0.76,0.048,0.77,0.088,1.7551,0.4368,DWARF,0.27101,0.00903,37.7647,0.0337,0.0,0.0,627,0.0186,,--,0.010943,0.0,0.0,,0.076,0.1,0.051,0.045,0.4186,0.455,0.0805,0.0925,0.00981,0.00825,0.0336,0.0337,bj2018,107.4,118.4,spect,9.74374,0.001652,8.46507,0.001891,1,hipvmag,apassdr9,,0.689515,0.697757,297.71017632661,48.08186352485,0.019271,0.024141,1,0,347873767
120571842,285.57397671045,38.40088256878,--,,643-065464,19021775+3824032,1237668734687445781,J190217.77+382402.9,2099925719951103872,--,3323887

Large queries (over one thousand objects) will be performed asynchronously and may take some time.  

Like `QueryPosition`, These results can also be returned as `astropy SkyCoord` objects.  

In [20]:
CatalogSearch.QueryID("TIC 377780790", return_skycoord=True)

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    [(285.67942179, 50.24130577, 186.50105373)]
 (pm_ra_cosdec, pm_dec) in mas / yr
    [(-18.394, 41.448)]>

In [21]:
result = CatalogSearch.QueryID("TIC 377780790")
result.to_SkyCoord()

<SkyCoord (ICRS): (ra, dec, distance) in (deg, deg, pc)
    [(285.67942179, 50.24130577, 186.50105373)]
 (pm_ra_cosdec, pm_dec) in mas / yr
    [(-18.394, 41.448)]>

## Looking up Catalog IDs

We have also included a limited way of looking up catalog IDs using other known IDs.  This functionality wraps the `astroquery.simbad.Simbad.query_objectids` functionality.  

In [22]:
CatalogSearch.IDLookup("Kepler 10")

id
object
LAMOST J190243.11+501428.7
AP J19024305+5014286
Gaia DR3 2132155017099178624
TIC 377780790
SPOCS 4268
Gaia DR2 2132155017099178624
Kepler-10
UCAC3 281-142262
2MASS J19024305+5014286


It is based off of joining IDs across Simbad tables, and is useful but not comprehensive as it relies upon published works to do the ID crossmatching.  That means that you may run into situations where not all assosciations have been made, eg. for this EPIC ID:

In [23]:
CatalogSearch.IDLookup("EPIC 201563164")

id
object
Gaia DR3 3796414192429498880
LBQS 1145+0145
WD 1145+017
SDSS J114833.62+012859.4
EPIC 201563164
HE 1145+0145
Gaia DR2 3796414192429498880


We can see above that this EPIC ID has a Gaia DR3 ID assosciated with it, and we know that GAIA DR3 and TIC are crossmatched, but the TIC ID is not reported.  To retrieve the TIC ID we can use the GAIA DR3 ID above and QueryID

In [24]:
CatalogSearch.QueryID("Gaia DR3 3796414192429498880", catalog="tic")

TIC,RAJ2000,DEJ2000,HIP,TYC,UCAC4,_2MASS,objID,WISEA,GAIA,APASS,KIC,S_G,Ref,r_Pos,pmRA,e_pmRA,pmDE,e_pmDE,r_pm,Plx,e_Plx,r_Plx,GLON,GLAT,ELON,ELAT,Bmag,e_Bmag,Vmag,e_Vmag,umag,e_umag,gmag,e_gmag,rmag,e_rmag,imag,e_imag,zmag,e_zmag,Jmag,e_Jmag,Hmag,e_Hmag,Kmag,e_Kmag,q_2MASS,W1mag,e_W1mag,W2mag,e_W2mag,W3mag,e_W3mag,W4mag,e_W4mag,Gmag,e_Gmag,Tmag,e_Tmag,f_Tmag,Flag,Teff,s_Teff,logg,s_logg,__M_H_,e__M_H_,Rad,s_Rad,Mass,s_Mass,rho,s_rho,LClass,Lum,s_Lum,Dist,s_Dist,E_B-V_,s_E_B-V_,Ncont,Rcont,Disp,m_TIC,Prior,e_E_B-V_,E_E_B-V_,f_E_B-V_,e_Mass,E_Mass,e_Rad,E_Rad,e_rho,E_rho,e_logg,E_logg,e_Lum,E_Lum,e_Dist,E_Dist,r_Dist,e_Teff,E_Teff,r_Teff,BPmag,e_BPmag,RPmag,e_RPmag,q_Gaia,r_Vmag,r_Bmag,Clist,e_RAJ2000,e_DEJ2000,RAOdeg,DEOdeg,e_RAOdeg,e_DEOdeg,RadFl,WDFl,ID
,deg,deg,,,,,,,,,,,,,mas / yr,mas / yr,mas / yr,mas / yr,,mas,mas,,deg,deg,deg,deg,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,mag,,,K,K,log(cm.s**-2),log(cm.s**-2),log(Sun),log(Sun),Rsun,Rsun,Msun,Msun,Sun,Sun,,Lsun,Lsun,pc,pc,mag,mag,,,,,,mag,mag,,Msun,Msun,Rsun,Rsun,Sun,Sun,log(cm.s**-2),log(cm.s**-2),Lsun,Lsun,pc,pc,,K,K,,mag,mag,mag,mag,,,,,mas,mas,deg,deg,mas,mas,,,
int64,float64,float64,int32,object,object,object,int64,object,int64,int32,int32,object,object,object,float64,float32,float64,float32,object,float64,float32,object,float64,float64,float64,float64,float32,float32,float32,float32,float32,float64,float32,float64,float32,float64,float32,float64,float32,float64,float32,float32,float32,float32,float32,float32,object,float32,float32,float32,float32,float32,float32,float32,float32,float64,float64,float32,float32,object,object,float32,float32,float32,float32,float32,float32,float64,float32,float32,float32,float64,float64,object,float64,float64,float64,float64,float32,float32,int32,float64,object,int64,float64,float32,float32,object,float32,float32,float32,float32,float64,float64,float32,float32,float64,float64,float64,float64,object,float32,float32,object,float64,float64,float64,float64,int16,object,object,object,float64,float64,float64,float64,float64,float64,int16,int16,int32
902906874,177.14012401504,1.48317268714,--,,,,--,,3796414192429498880,--,--,STAR,gaia2,gaia2,-43.71,0.242,-4.102,0.134,gaia2,7.0585,0.1234,gaia2,269.85004856685,60.25223135972,176.78646975797,0.22389926894,--,--,17.19,0.046,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,--,,--,--,--,--,--,--,--,--,17.1671,0.001697,17.33,0.012,rered,,--,--,--,--,--,--,--,--,--,--,--,--,,--,--,141.166,2.4785,0.0146,0.0094,--,--,,--,--,0.0066,0.0122,panstarrs,--,--,--,--,--,--,--,--,--,--,2.436,2.521,bj2018,--,--,,17.0393,0.012083,17.2342,0.009613,1,gaia2,,,3.759826,2.071664,177.13993575829,1.48315502679,0.115832,0.070643,-1,1,144272311


`lksearch` also alows you to input a list of strings, which will then perform sequential ID searches for each item in the list and return a list of the resulting tables: 

In [25]:
CatalogSearch.IDLookup(["Kepler 10", "Kepler 16"])

Throttling query limit to Simbad's: max 5/s


[<Table length=14>
              id             
            object           
 ----------------------------
   LAMOST J190243.11+501428.7
         AP J19024305+5014286
 Gaia DR3 2132155017099178624
                TIC 377780790
                   SPOCS 4268
 Gaia DR2 2132155017099178624
                    Kepler-10
             UCAC3 281-142262
      2MASS J19024305+5014286
        PPMX J190243.0+501428
      USNO-B1.0 1402-00324696
              GSC 03549-00354
                       KOI-72
                 KIC 11904151,
 <Table length=11>
              id             
            object           
 ----------------------------
   LAMOST J191618.16+514526.7
 Gaia DR3 2133476355197071616
                TIC 299096355
 Gaia DR2 2133476355197071616
      2MASS J19161817+5145267
             UCAC3 284-140854
              GSC 03554-01147
                 KIC 12644769
                    Kepler-16
          NAME Kepler-16 (AB)
                     KOI-1611]

### Matching ID Results to Catalogs
`lksearch` has the ability to parse the id lists returned by `QueryID` to look for ids which contain the string contained in the optional parameter `match`.  This can be usefull for only returning the catalog results which interest you. 

Just like the search input can be a list, so can the match criteria, as so: 

In [26]:
CatalogSearch.IDLookup(["Kepler 10", "Kepler 16"], match=["tic", "kic"])

Throttling query limit to Simbad's: max 5/s


search,tic,kic
str9,str13,str12
Kepler 10,TIC 377780790,KIC 11904151
Kepler 16,TIC 299096355,KIC 12644769
